In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorrt
import tensorflow as tf
from pathlib import Path

2023-06-10 17:23:11.164077: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
tf.config.list_physical_devices()

2023-06-10 17:23:32.623990: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [4]:
cifar10 = tf.keras.datasets.cifar10.load_data()
(X_train_full, y_train_full), (X_test, y_test) = cifar10

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

In [5]:
tf.random.set_seed(42)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(tf.keras.layers.Dense(100, kernel_initializer="he_normal"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation("swish"))

model.add(tf.keras.layers.Dense(10, activation="softmax"))

optimizer = tf.keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=20,
                                                     restore_best_weights=True)
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_cifar10_bn_model",
                                                         save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = Path() / "my_cifar10_logs" / f"run_bn_{run_index:03d}"
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

model.evaluate(X_valid, y_valid)

Epoch 1/100
1407/1407 [==============================] - ETA: 0s - loss: 2.0571 - accuracy: 0.2514

INFO:tensorflow:Assets written to: my_cifar10_bn_model/assets


INFO:tensorflow:Assets written to: my_cifar10_bn_model/assets


1407/1407 [==============================] - 35s 18ms/step - loss: 2.0571 - accuracy: 0.2514 - val_loss: 2.0840 - val_accuracy: 0.2550
Epoch 2/100
1407/1407 [==============================] - 19s 14ms/step - loss: 1.7934 - accuracy: 0.3581 - val_loss: 2.2973 - val_accuracy: 0.2450
Epoch 3/100
1406/1407 [============================>.] - ETA: 0s - loss: 1.6861 - accuracy: 0.3976

INFO:tensorflow:Assets written to: my_cifar10_bn_model/assets


...

1407/1407 [==============================] - 26s 18ms/step - loss: 1.3972 - accuracy: 0.5060 - val_loss: 1.4980 - val_accuracy: 0.4700
Epoch 10/100
1407/1407 [==============================] - 19s 14ms/step - loss: 1.3648 - accuracy: 0.5156 - val_loss: 1.6268 - val_accuracy: 0.4224
Epoch 11/100
1407/1407 [==============================] - 19s 13ms/step - loss: 1.3399 - accuracy: 0.5242 - val_loss: 1.5600 - val_accuracy: 0.4552
Epoch 12/100
1407/1407 [==============================] - 19s 13ms/step - loss: 1.3063 - accuracy: 0.5378 - val_loss: 1.5927 - val_accuracy: 0.4418
Epoch 13/100
1407/1407 [==============================] - 19s 13ms/step - loss: 1.2812 - accuracy: 0.5450 - val_loss: 1.5089 - val_accuracy: 0.4804
Epoch 14/100
1407/1407 [==============================] - 19s 14ms/step - loss: 1.2577 - accuracy: 0.5544 - val_loss: 1.5159 - val_accuracy: 0.4692
Epoch 15/100
1407/1407 [==============================] - 17s 12ms/step - loss: 1.2333 - accuracy: 0.5635 - val_loss: 1.5075 

INFO:tensorflow:Assets written to: my_cifar10_bn_model/assets


INFO:tensorflow:Assets written to: my_cifar10_bn_model/assets


1407/1407 [==============================] - 27s 19ms/step - loss: 1.1081 - accuracy: 0.6082 - val_loss: 1.4482 - val_accuracy: 0.5038
Epoch 23/100
1407/1407 [==============================] - 20s 14ms/step - loss: 1.0928 - accuracy: 0.6157 - val_loss: 1.4814 - val_accuracy: 0.4988
Epoch 24/100
1407/1407 [==============================] - 19s 14ms/step - loss: 1.0725 - accuracy: 0.6222 - val_loss: 1.5115 - val_accuracy: 0.4950
Epoch 25/100
1407/1407 [==============================] - 18s 13ms/step - loss: 1.0607 - accuracy: 0.6242 - val_loss: 1.5644 - val_accuracy: 0.4742
Epoch 26/100
1407/1407 [==============================] - 17s 12ms/step - loss: 1.0486 - accuracy: 0.6300 - val_loss: 1.4981 - val_accuracy: 0.4916
Epoch 27/100
1407/1407 [==============================] - 17s 12ms/step - loss: 1.0360 - accuracy: 0.6351 - val_loss: 1.5945 - val_accuracy: 0.4628
Epoch 28/100
1407/1407 [==============================] - 17s 12ms/step - loss: 1.0247 - accuracy: 0.6380 - val_loss: 1.5426 

[1.448202133178711, 0.5037999749183655]